# Diabetes Dataset - EDA

In this notebook, an initial Exploratory Data Analysis (EDA) is performed, followed by data cleaning and the plotting of insights related to the data contained in the dataset `diabetes_unclean.csv`.

## Opening and First look
In this section, the necessary libraries are imported, and a preliminary analysis of the dataset is performed.

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
dataset = pd.read_csv('../datasets/diabetes_unclean.csv')

In [ ]:
dataset.head()

In [ ]:
print(dataset.columns.values)

## Dataset features
In the dataset we can access the following feature:

- **Gender**: The gender of the individual (e.g., Male, Female).
- **AGE**: The age of the individual in years.
- **Urea**: The level of urea in the blood, indicating kidney function. 
- **Cr**: Creatinine level in the blood, used to assess kidney function.
- **HbA1c**: Hemoglobin A1c percentage, a measure of average blood sugar levels.
- **Chol**: Total cholesterol level in the blood, measured in mmol/L, indicating lipid profile.
- **TG**: Triglycerides level in the blood, measured in mmol/L, another component of the lipid profile.
- **HDL**: High-Density Lipoprotein cholesterol, often referred to as "good cholesterol," measured in mmol/L.
- **LDL**: Low-Density Lipoprotein cholesterol, often referred to as "bad cholesterol," measured in mmol/L.
- **VLDL**: Very Low-Density Lipoprotein cholesterol, another type of "bad cholesterol," measured in mmol/L.
- **BMI**: Body Mass Index, a measure of body fat based on height and weight.
- **CLASS**: The target variable indicating the presence or absence of diabetes (e.g., diabetic or non-diabetic).

In [ ]:
dataset.info()


In [ ]:
dataset.describe()

## Data Cleaning and Preprocessing
In this section, data cleaning is performed by analyzing entries that are outside the acceptable range or are potential outliers. Additionally, preprocessing and encoding of non-numeric or categorical features are carried out.


List of task to be performed in this section:
- Backup the original dataset
- Check for Null values
- Conversion to numerical values or one-hot encoding of the 'object' or 'string' types
- Check for outliers and cleaning


### Backup of the dataset

In [ ]:
ds_backup = dataset.copy()
ds_backup.to_csv('../datasets/diabetes_unclean_backup.csv', index=False) # This saves a backup of the original dataset

In [ ]:
# counting the number of missing values in each column
dataset.isnull().sum()

### Null values 
Since the total entries of the dataset are 1009 and the Null values are less than the 2% of the dataset it is possible to remove them (no fill required)


In [ ]:
# Removing the rows with missing values and applying the changes to the dataset
dataset.dropna(inplace=True)

### Conversion to numerical values

In [ ]:
dataset.dtypes

Columns to be converted or encoded:
- Gender -> simple encoding [0 = 'M', 1 = 'F']
- Class -> simple encoding [0 = non-positive, 1 = positive]


In [ ]:
dataset['Gender'].unique()

As there are inconsistencies in the gender column we will perform a 'normalization' of the data and a conversion to numeric value 
```
[0: Male, 1: Female]
```

In [ ]:
# Repairing inconsistent values and Encoding of the 'Gender' column
dataset['Gender'] = dataset['Gender'].str.upper().map({'M': 0, 'F': 1})

In [ ]:
dataset['CLASS'].unique()

We can see that there are several unique values with inconsistent syntax, since the possible value of the column `CLASS` are 'Diabetic' or 'Not Diabetic' we will encode it with the following syntax:
```
[Positive to diabetes: 1]
[Negative to diabetes: 0]
```

In [ ]:
# Encoding of the 'CLASS' column
# 1. remove unwanted spaces
dataset['CLASS'] = dataset['CLASS'].str.strip()
# 2. raise all values to upper case
dataset['CLASS'] = dataset['CLASS'].str.upper()
# 3. make the values consistent (e.g. P means positive as well as Y, instead, N means negative or simply No)
dataset['CLASS'] = dataset['CLASS'].replace({'P': 'Y'})

# check:
# dataset['CLASS'].unique()

# 4. Encoding of the 'CLASS' column
dataset['CLASS'] = dataset['CLASS'].map({'Y': 1, 'N': 0})

### Unused columns
Some columns are not relevant to predict diabetes. For example, columns like `ID` and `No_Pation` do not provide meaningful information for prediction and can be removed.

In [ ]:
# removing unused columns
# backup of the cleaned and encoded dataset
ds_backup = dataset.copy()
# removing unused columns
dataset.drop(columns=['ID', 'No_Pation'], inplace=True)


In [ ]:
# Count the occurrences of each class
class_counts = dataset['CLASS'].value_counts()

# Calculate the percentage
class_percentages = (class_counts / len(dataset)) * 100

# Print the results
print("Diabetes (CLASS = 1):")
print(f"Count: {class_counts[1]}, Percentage: {class_percentages[1]:.2f}%")
print("\nNo Diabetes (CLASS = 0):")
print(f"Count: {class_counts[0]}, Percentage: {class_percentages[0]:.2f}%")

### Pairplot of the features to get an overall view of the data

In [ ]:
sns.pairplot(dataset, diag_kind='kde', hue='CLASS')


### Outliers

In this section, we identify outliers to ensure the dataset is consistent, enabling accurate analysis and reliable predictions using future models.

In [ ]:
stats = dataset.drop(columns=['Gender', 'AGE']).describe() 
# storing the stats in a dataframe (excluding gender and age columns)
stats

In [ ]:
'''
Filtering columns with a standard deviation greater than half the mean and store it in a list
(retrieved from the just made 'stats' dataframe) 
'''


# Boolean mask to filter columns: 
high_std_mask = stats.loc['std'] > (stats.loc['mean'] / 2)
# Apply the mask and create a list of 'suspicious' features:
features_high_std = stats.columns[high_std_mask] 
suspicious_features = list(features_high_std)
print(suspicious_features)

In [ ]:
# Plotting the distribution of the features as subplots horizontally

plt.figure(figsize=(15, 20))

# Loop through each feature and create a subplot: 
# i is the index of the feature in the dataset, feature is the name of the feature
for i, feature in enumerate(dataset.columns, 1):
    plt.subplot(len(dataset.columns), 1, i)         # Arrange subplots vertically
    sns.boxplot(data=dataset[feature], orient='h')  # Set orientation to horizontal
    plt.title(f'Boxplot of {feature}')
    plt.xlabel(feature)

plt.tight_layout()
plt.show()


# Outlier Detection Analysis

## Features Requiring Special Attention

As shown in the preliminary analysis, the features **'Urea', 'Cr', 'TG', 'HDL',** and **'VLDL'** show high values of standard deviation ($\sigma$). Specifically, these columns have standard deviation values higher than half their respective means:

$$\text{Attention required when:}\quad \sigma > \frac{\text{mean}}{2}$$

This indicates potential outliers that warrant further investigation.

## Physiological Limits for Outlier Detection

Physiological ranges and limits:

| Feature | Lower Limit | Upper Limit | Justification |
|---------|-------------|-------------|---------------|
| **Urea** | 1 mmol/L | 25 mmol/L | Values outside this range are extremely rare in living patients and likely represent measurement errors |
| **Cr (Creatinine)** | 10 µmol/L | 400 µmol/L | Values above 400 µmol/L may indicate severe renal failure but could also be data entry errors; values below 10 µmol/L are biologically implausible |
| **TG (Triglycerides)** | 0.1 mmol/L | 10 mmol/L | While normal range is <1.7 mmol/L, values up to 10 mmol/L can occur in severe hypertriglyceridemia or **diabetic** patients |
| **HDL** | 0.3 mmol/L | 5 mmol/L | Values above 5 mmol/L are highly improbable and likely due to laboratory errors or data entry mistakes |
| **VLDL** | 0.05 mmol/L | ? mmol/L | Value derived from a medical equation that considers the levels of TG and cholesterol in the blood. Further investigation needed, possible syntethic values not needed for this project. |


  *! Diclaimer*: The reference values provided are not intended for medical purposes and were obtained through online research. They are not guaranteed to be reliable or representative of the sample in question, as this is not the primary objective of the project.
  Sources: https://www.scymed.com, https://www.my-personaltrainer.it/salute/conversione-colesterolo.html




#### Box-plot of the suspicious feature:
In the box-plot above, it's evident that features such as 'Urea', 'Cr', 'TG', 'HDL', and 'VLDL' exhibit substantial variance with an asymmetric skew towards the upper extremes. However, we can't straightforwardly discard data above the third quartile since these patients might be suffering from diabetes-related pathologies or severe dysfunctions, information that could be needed in the prediction model and further analysis.

In [ ]:
# setting and storing in lists the upper and lower limits for each suspicious feature:
#                   Urea, Cr,  TG, HDL, VLDL
temp_limits_upper = [25,  400, 10,  5,   40 ]
temp_limits_lower = [1,   10,  0.1, 0.3, 0.05]

# converting it to a dataframe for better readability:
suspicious_features_boundaries = pd.DataFrame({'Feature': suspicious_features, 'Upper Limit': temp_limits_upper, 'Lower Limit': temp_limits_lower})

suspicious_features_boundaries

#### Identification of the values that exceed the biological plausibility thresholds and removal

In [ ]:
'''
plotting the same boxplots for the suspicious features adding the upper and lower limits
for graphical reference:
'''
plt.figure(figsize=(15, 10))

data = dataset[features_high_std]

# Loop through each feature and create a subplot
for i, feature in enumerate(features_high_std, 1):
    plt.subplot(len(features_high_std), 1, i)  # Arrange subplots vertically
    sns.boxplot(data=data[feature], orient='h')  # Set orientation to horizontal
    plt.title(f'Boxplot of {feature}')
    plt.xlabel(feature)
    
    # Red line for the upper threshold
    plt.axvline(x=temp_limits_upper[i - 1], color='red', linestyle='-', label='Threshold')
    
    # Ged line for the lower threshold
    if temp_limits_lower[i - 1] > 0:
           plt.axvline(x=temp_limits_lower[i - 1], color='green', linestyle='-', label='Threshold') 

    plt.legend()

plt.tight_layout()
plt.show()


Computing the number of data-points above the thresholds for each abnormal variance column.

In [ ]:
# Add two new columns to the suspicious_features_boundaries DataFrame
suspicious_features_boundaries['Above upper limits'] = [
    data[feature][data[feature] > suspicious_features_boundaries.loc[i, 'Upper Limit']].count()
    for i, feature in enumerate(suspicious_features_boundaries['Feature'])
]

suspicious_features_boundaries['Below lower limits'] = [
    data[feature][data[feature] < suspicious_features_boundaries.loc[i, 'Lower Limit']].count()
    for i, feature in enumerate(suspicious_features_boundaries['Feature'])
]

# Display the DataFrame
suspicious_features_boundaries

#### Removing or replacing outliers

For now, we replace the outliers with NaN values. This approach allows us to retain the flexibility to either remove these entries or handle them differently in subsequent steps, depending on the requirements of the analysis.



In [ ]:
# replacing with NaN

for index, row in suspicious_features_boundaries.iterrows():
    feature = row['Feature']
    upper_limit = row['Upper Limit']
    lower_limit = row['Lower Limit']
    dataset.loc[(dataset[feature] > upper_limit) | (dataset[feature] < lower_limit), feature] = np.nan

# counting the number of missing values in each column
dataset.isnull().sum()

### VLDL and TG
Since VLDL is a value typically synthetic, we investigate its distribution in relation to TG. The scatter plot below highlights their relationship, showing a positive correlation between the two variables.

VLDL can also be measured directly in some cases, thus the relation with the TG value can be non-linear in some cases.

Formulas for computing VLDL starting from TG:
$$VLDL = \frac{\text{TG}}{5} \quad \text{(mg/dL)}$$ 
$$VLDL = \frac{\text{TG}}{2.2} \quad \text{(mmol/L)}$$

#### Conversion rate between different units of measurement:
- (TG):

$$\text{mmol/L} = \frac{\text{mg/dL}}{88.5}$$
$$\text{mg/dL} = \text{mmol/L} \times 88.5$$

- Cholesterol (LDL, HDL, VLDL, Totale):

$$\text{mmol/L} = \frac{\text{mg/dL}}{38.67}$$
$$\text{mg/dL} = \text{mmol/L} \times 38.67$$


In [ ]:
# Scatter plot for TG vs VLDL with color based on threshold
plt.figure(figsize=(15, 10))

# Create a standalone pandas Series to classify points based on the threshold
VLDL_thresh_status = pd.Series(np.where(dataset['VLDL'] > 4, 'above', 'below'), index=dataset.index)



# Scatter plot with color coding
sns.scatterplot(data=dataset, x='TG', y='VLDL', hue=VLDL_thresh_status, alpha=0.5)

plt.title('Scatter Plot: TG vs VLDL')
plt.xlabel('TG')
plt.ylabel('VLDL')
plt.axhline(y=4, color='red', linestyle='--', label='Hypothesis Threshold')
plt.legend()
plt.grid(True)
plt.show()



In [ ]:
# Filter the dataset for VLDL values under 4
filtered_dataset = dataset[dataset['VLDL'] < 4]

# Scatter plot for TG vs VLDL with VLDL < 4
plt.figure(figsize=(15, 10))
sns.scatterplot(data=filtered_dataset, x='TG', y='VLDL', alpha=0.5, hue=VLDL_thresh_status)
plt.title('Scatter Plot: TG vs VLDL (VLDL < 4)')
plt.xlabel('TG')
plt.ylabel('VLDL')

# Plot the line y = x / 2.2
x_vals = np.linspace(filtered_dataset['TG'].min(), filtered_dataset['TG'].max(), 100)
y_vals = x_vals / 2.2
plt.plot(x_vals, y_vals, color='red', label='VLDL = TG / 2.2')
plt.legend()

plt.grid(True)

plt.show()

In [ ]:
# Filter the dataset for VLDL values over 4
filtered_dataset = dataset[dataset['VLDL'] >= 4]

# Scatter plot for TG vs VLDL with VLDL >= 4
plt.figure(figsize=(15, 10))
sns.scatterplot(data=filtered_dataset, x='TG', y='VLDL', alpha=0.5, hue=VLDL_thresh_status)
plt.title('Scatter Plot: TG vs VLDL (VLDL >= 4)')
plt.xlabel('TG')
plt.ylabel('VLDL')
plt.grid(True)

x_vals = np.linspace(filtered_dataset['TG'].min(), filtered_dataset['TG'].max(), 100)
y_vals = x_vals * 38.67 / 5.5
plt.plot(x_vals, y_vals, color='red', label='VLDL = TG * 38.67 / 5.5')
plt.legend()

# VLDL = TG * 38.67 / 5.5
# TG = VLDL * 5.5 / 38.67
# converted_VLDL = (VLDL * 5.5 / 38.67) / 2.2



plt.show()

### There are two groups of VLDL values, as seen in the graphs above. 

- One group consists of values less than 4, following a pseudo-linear distribution composed of synthetic values derived from the standard formula ($VLDL = TG / 2.2$) with the unit of measurement: mmol/L. 
- The other group exhibits an apparently incorrect trend. Further analysis has demonstrated that the second group represents measurements in a different unit of measurement (mg/dL). 

To homogenize the data, it will be necessary to convert the VLDL measurements back to TG and re-compute the VLDL value as in the other group:  


In [ ]:
# Convert VLDL values greater than 4 using the new formula
dataset.loc[dataset['VLDL'] > 4, 'VLDL'] = (dataset['VLDL'] * 5.5 / 38.67) / 2.2

# Scatter plot for TG vs VLDL after conversion
plt.figure(figsize=(15, 10))
sns.scatterplot(data=dataset, x='TG', y='VLDL', alpha=0.5)
plt.title('Scatter Plot: TG vs VLDL (After Conversion)')
plt.xlabel('TG')
plt.ylabel('VLDL')

# Plot the line y = x / 2.2
x_vals = np.linspace(dataset['TG'].min(), dataset['TG'].max(), 100)
y_vals = x_vals / 2.2
plt.plot(x_vals, y_vals, color='red', label='VLDL = TG / 2.2')
plt.legend()

plt.grid(True)
plt.show()


In [ ]:
# Count the number of rows with one or more NaN values
rows_with_nan = dataset.isnull().any(axis=1).sum()

# Calculate the percentage of rows with NaN values
percentage_with_nan = (rows_with_nan / len(dataset)) * 100

print(f"Number of rows with NaN values: {rows_with_nan}")
print(f"Percentage of rows with NaN values: {percentage_with_nan:.2f}%")

In [ ]:
dataset.dropna(inplace=True)

# Saving the cleaned dataset

In [ ]:
dataset.to_csv('../datasets/diabetes_cleaned.csv', index=False)